# Example IMC analysis with Morpheus

## Step 0 (optional): set seed for reproducibility 

In [1]:
from lightning.pytorch import seed_everything
seed_everything(42)

%reload_ext autoreload
%autoreload 2

Seed set to 42


## Step 1: Creating a SpatialDataset Object

In this tutorial, we will start by creating a `SpatialDataset` object, which will hold all relevant information about the dataset we will be working with. 

### Prerequisites

To create a `SpatialDataset` object, you will need:
- The path to the CSV file containing all single-cell expression information
- A list of channel names

### CSV File Structure

The expected structure of the CSV file is as follows:
- Each row corresponds to a single cell
- Columns for each channel name, with expression values specified
- Five additional columns with the following names and information:

| Column Name         | Description                               |
|---------------------|-------------------------------------------|
| `ImageNumber`       | Unique ID for each image                  |
| `PatientID`         | Unique ID for each patient                |
| `CellType`          | Cell type                                 |
| `Location_Center_X` | X coordinate of the cell center in micron |
| `Location_Center_Y` | Y coordinate of the cell center in micron |

Note: Additional columns beyond these will not be used for the analysis performed in this tutorial.

To create a `SpatialDataset` object, use the following code, remember to replace `'path/to/your/csv/file.csv'` with the actual path to your CSV file.

In [2]:
import morpheus as mp

data_path = "/groups/mthomson/zwang2/IMC/output/cedarsLiver_sz48_pxl3_nc44/temp/singlecell.csv"  # change to your own directory
dataset = mp.SpatialDataset(input_path=data_path)

2024-04-07 00:31:56.904960: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-07 00:32:09.547885: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /central/home/zwang2/anaconda3/lib/python3.8/site-packages/nvidia/cudnn/:/central/slurm/install/current/lib/:
2024-04-07 00:32:09.548524: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /central/home/zwang2

Channel names inferred from input CSV: ['CD45', 'Glnsynthetase', 'CD163', 'NKG2D', 'CCR4', 'PDL1', 'FAP', 'CD11c', 'LAG3', 'HepPar1', 'FOXP3', 'aSMA', 'CD4', 'CD105endoglin', 'CD68', 'VISTA', 'CD20', 'CD8a', 'TIM3', 'CXCR4', 'PD1', 'iNOS', 'CD31', 'CYR61', 'CDX2', 'CAIX', 'CD3', 'CD44', 'CD15', 'CD11b', 'HLADR', 'IL10', 'CXCL12', 'HLAABC', 'DNA1', 'DNA2', 'GranzymeB', 'Ki67', 'HistoneH3', 'CXCR3', 'Galectin9', 'YAP', 'CD14', 'CK19']


## Step 2: patch images and mask cells

In [3]:
patch_size = 16  # Patch size in pixels
pixel_size = 3  # Pixel size in microns
cell_types = ["Tcytotoxic", "Tumor"]  # Specify the cell types of interest
mask_cell_types = ["Tcytotoxic"]
dataset.generate_masked_patch(
    cell_to_mask=mask_cell_types,
    cell_types=cell_types,
    patch_size=patch_size,
    pixel_size=pixel_size,
    save=True,
)

# example metadata
print(dataset.metadata.head())

File /groups/mthomson/zwang2/IMC/output/cedarsLiver_sz48_pxl3_nc44/temp/patch.h5 already exists, not saving patches
   patch_id  ImageNumber  PatientID  PatchIndex_X  PatchIndex_Y  \
0         0            2          2             0             0   
1         1            2          2             0             1   
2         2            2          2             0             2   
3         3            2          2             0             3   
4         4            2          2             0             4   

   Contains_Tcytotoxic  Contains_Tumor  
0                False            True  
1                 True            True  
2                False            True  
3                 True           False  
4                 True           False  


## Step 3: generate data splits for model training

Next, we will need to generate train, validation, and test data splits for model training. We want to stratify our splits by the label we want to predict.

In [4]:
colname = "Contains_Tcytotoxic"
dataset.generate_data_splits(stratify_by=colname)

Data splits already exist in /groups/mthomson/zwang2/IMC/output/cedarsLiver_sz48_pxl3_nc44/temp/split


## Step 4: train classifier model

In [ ]:
# initialize model
model_arch = "unet"
n_channels = dataset.n_channels
img_size = dataset.img_size
model = mp.PatchClassifier(n_channels, img_size, model_arch)

# train model
trainer_params = {
    "max_epochs": 100,  # set to >60 for better performance
    "accelerator": "auto",
    "logger": False,
}
model = mp.train(
    model=model,
    dataset=dataset,
    label_name=colname,
    trainer_params=trainer_params,
)

## Step 5: generate counterfactuals using trained classifier

In [26]:
# load model if needed
model = mp.load_model(dataset.model_dir + "/checkpoints/epoch=41-step=12432.ckpt")

# images to generate counterfactuals
dataset.get_split_info()
select_metadata = dataset.metadata[
    (dataset.metadata["Contains_Tumor"] == 1)
    & (dataset.metadata["Contains_Tcytotoxic"] == 0)
    & (dataset.metadata["splits"] == "train")
    & (dataset.metadata["ImageNumber"] == 202)
]

Using cache found in /home/zwang2/.cache/torch/hub/mateuszbuda_brain-segmentation-pytorch_master


In [44]:
# channels allowed to be perturbed
channel_to_perturb = [
    "Glnsynthetase",
    "CCR4",
    "PDL1",
    "LAG3",
    "CD105endoglin",
    "TIM3",
    "CXCR4",
    "PD1",
    "CYR61",
    "CD44",
    "IL10",
    "CXCL12",
    "CXCR3",
    "Galectin9",
    "YAP",
]

# probability cutoff for classification
threshold = 0.43

# optimization parameters
optimization_param = {
    "use_kdtree": True,
    "theta": 40.0,
    "kappa": -0.14,  # set to: (threshold - 0.5) * 2
    "learning_rate_init": 0.1,
    "beta": 1.0,
    "max_iterations": 1000,  # set to >1000 for better performance
    "c_init": 1000.0,
    "c_steps": 5,
}

# Generate counterfactuals using trained model
cf = mp.get_counterfactual(
    images=select_metadata.sample(frac=1, random_state=40),
    dataset=dataset,
    target_class=1,
    model=model,
    channel_to_perturb=channel_to_perturb,
    optimization_params=optimization_param,
    threshold=threshold,
    verbose=False,
)

  0%|          | 1/209 [02:35<8:58:06, 155.23s/it]

Counterfactual probability: [0.28606406 0.7139359 ]
cf perturbed: {'Glnsynthetase': 8.732989e-06, 'CCR4': -100.0, 'PDL1': -45.725613, 'LAG3': 6.373288e-06, 'CD105endoglin': 0.0, 'TIM3': 0.0, 'CXCR4': -100.0, 'PD1': -88.67476, 'CYR61': -91.752975, 'CD44': 9.093891e-06, 'IL10': 0.0, 'CXCL12': -100.0, 'CXCR3': 0.0, 'Galectin9': -9.089886e-06, 'YAP': 0.0}


  1%|          | 2/209 [05:09<8:53:18, 154.58s/it]

  1%|▏         | 3/209 [07:45<8:53:33, 155.41s/it]

Counterfactual probability: [0.48096082 0.51903915]
cf perturbed: {'Glnsynthetase': 0.0, 'CCR4': -7.966883e-06, 'PDL1': -8.6221435e-06, 'LAG3': 8.926197e-06, 'CD105endoglin': 0.0, 'TIM3': 1.1869174e-05, 'CXCR4': -60.871284, 'PD1': -22.581678, 'CYR61': 0.0, 'CD44': -0.6644316, 'IL10': 0.0, 'CXCL12': 0.0, 'CXCR3': -8.007229e-06, 'Galectin9': 0.0, 'YAP': -6.0159678e-06}


...

Counterfactual probability: [0.01054065 0.9894594 ]
cf perturbed: {'Glnsynthetase': 9.568526e-06, 'CCR4': -100.0, 'PDL1': -100.0, 'LAG3': 1.1485368e-05, 'CD105endoglin': 0.0, 'TIM3': -100.0, 'CXCR4': -100.0, 'PD1': -99.03589, 'CYR61': -76.673546, 'CD44': 0.0, 'IL10': 0.0, 'CXCL12': 0.0, 'CXCR3': -6.3989214e-06, 'Galectin9': -6.406129e-06, 'YAP': 74.317924}


 57%|█████▋    | 120/209 [4:38:47<3:41:13, 149.14s/it]

Counterfactual probability: [0.09785805 0.9021419 ]
cf perturbed: {'Glnsynthetase': 0.0, 'CCR4': -100.0, 'PDL1': -83.63405, 'LAG3': -32.378475, 'CD105endoglin': 0.0, 'TIM3': -27.352041, 'CXCR4': -100.0, 'PD1': -46.610443, 'CYR61': -53.75555, 'CD44': 0.0, 'IL10': 0.0, 'CXCL12': -100.0, 'CXCR3': -1.1462672e-05, 'Galectin9': 9.202838e-06, 'YAP': 0.0}


 58%|█████▊    | 121/209 [4:41:22<3:41:06, 150.76s/it]

Counterfactual probability: [8.3021499e-05 9.9991703e-01]
cf perturbed: {'Glnsynthetase': 0.0, 'CCR4': -100.0, 'PDL1': -63.476677, 'LAG3': 69.52631, 'CD105endoglin': 163.5228, 'TIM3': -100.0, 'CXCR4': -100.0, 'PD1': -61.37461, 'CYR61': -96.16943, 'CD44': -29.845259, 'IL10': -100.0, 'CXCL12': -96.825516, 'CXCR3': 0.0, 'Galectin9': 0.0, 'YAP': 88.073235}
